<a href="https://colab.research.google.com/github/sarikasea/Agentic-RAG-with-Llamaindex/blob/main/OpenAI_L1_Router_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
#%pip install -r requirements.txt

In [2]:
pip install llama-index-llms-openai llama-index-embeddings-openai

In [3]:
pip install openai>=1.0.0

In [4]:
pip install llama-index

In [5]:
!pip show llama-index llama-index-llms-openai

Name: llama-index
Version: 0.14.7
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 
---
Name: llama-index-llms-openai
Version: 0.6.7
Summary: llama-index llms openai integration
Home-page: 
Author: llama-index
Author-email: 
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: llama-index-core, openai
Required-by: llama-index, llama-index-cli


In [6]:
%pip install --upgrade llama-index llama-index-llms-openai

In [14]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

ImportError: cannot import name 'get_openai_api_key' from 'helper' (unknown location)

In [13]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# from openai import OpenAI

# client = OpenAI()

# try:
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": "Hello world!"},
#         ]
#     )
#     print("OpenAI API connection successful!")
#     print(response.choices[0].message.content)
# except Exception as e:
#     print(f"Error connecting to OpenAI API: {e}")

In [15]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [17]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [18]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [19]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-4-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [20]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [21]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [22]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [23]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [24]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Choice 1 is directly related to summarization, which is relevant to the question asking for a summary of the document..
The document describes MetaGPT, a meta-programming framework designed to enhance multi-agent collaborations using large language models (LLMs). MetaGPT integrates standardized operating procedures (SOPs) into its system, allowing for structured communication and role specialization among agents. This framework aims to address issues such as logic inconsistencies and cascading hallucinations that arise in complex task environments. MetaGPT has shown improved performance in software engineering benchmarks, demonstrating its ability to generate coherent and robust solutions. The framework also features an executable feedback mechanism to further refine code generation during runtime. The document details the structure, functionality, and evaluation of MetaGPT, highlighting its advantages over existing LLM-based multi-agent systems.


In [25]:
print(len(response.source_nodes))

34


In [26]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is relevant because it involves retrieving specific context from the MetaGPT paper, which is likely necessary to answer a detailed question about how agents share information..
Agents share information with other agents through a shared message pool where they publish structured messages. They can also subscribe to relevant messages based on their profiles, allowing them to access messages from other entities transparently. This system enables agents to directly retrieve required information from the pool, eliminating the need for direct inquiries and waiting for responses from other agents.


## Let's put everything together

In [28]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [29]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: This choice is relevant because retrieving specific context from the MetaGPT paper, such as details about the ablation study results, requires accessing particular information within the document..
The ablation study results provide insights into the impact of different components or features of a system by systematically removing them and observing the effects on the overall performance. This type of study helps in understanding the individual contributions of various elements towards the system's functionality or effectiveness.
